# Carregamento de Dados

In [20]:
import pandas as pd
from ipywidgets import interact
import os
from pyarrow.parquet import ParquetFile

BATCH_SIZE=64000
ds = {
    'associates': ParquetFile('../data/parquet/associados.parquet').iter_batches(batch_size=BATCH_SIZE),
    'pdvs': ParquetFile('../data/parquet/pdvs.parquet').iter_batches(batch_size=BATCH_SIZE),
    'products': ParquetFile('../data/parquet/produtos.parquet').iter_batches(batch_size=BATCH_SIZE)
}


#@interact
#def set_ds(file=os.listdir('../data/parquet/')):
#    path = f'../data/parquet/{file}'
#    # Batch loading based on https://stackoverflow.com/questions/53982871/pandas-reading-first-n-rows-from-parquet-file
#    parquet = ParquetFile(path)
#    ds['size'] = parquet.scan_contents()
#    ds['data'] = parquet.iter_batches(batch_size=BATCH_SIZE)
    #display(next(ds['data']).to_pandas())

#next(ds['associates']).to_pandas()


# Qtd. de Produtos por PDV/SKU ao longo do Tempo

- products: produto, categoria, descricao, tipos, label, subcategoria, marca, fabricante
- pdvs: pdv, premise, categoria_pdv, zipcode
- associates: internal_store_id, internal_product_id, distributor_id, transaction_date, reference_date, quantity, gross_value, net_value, gross_profit, discount, taxes

In [ ]:
import numpy as np


# Objetivo do modelo: saber a quantidade de vendas prevista de um produto, dado o local (loja)

# Para cada trecho dos dados
for batch in ds['associates']:
    # Modificá-los
    df = batch.to_pandas()
    display(df[ (df['internal_product_id'] == '328903483604537190') ])

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.95,0.234375
1275,838077005719258266,328903483604537190,4,2022-06-15,2022-06-01,1.0,35.599998,32.631565,10.565565,0.00,2.968435
3190,7508866696679644291,328903483604537190,5,2022-06-22,2022-06-01,1.0,39.650002,39.415627,10.351626,2.70,0.234375
3391,3927635916396784077,328903483604537190,7,2022-12-01,2022-12-01,1.0,41.365002,41.130627,10.582626,3.75,0.234375
4994,438520484720188884,328903483604537190,4,2022-11-14,2022-11-01,1.0,38.200001,35.231567,10.050568,0.00,2.968435
...,...,...,...,...,...,...,...,...,...,...,...
57214,7761595785944224276,328903483604537190,5,2022-01-18,2022-01-01,1.0,39.650002,39.415627,11.121627,2.70,0.234375
57481,2891253628316038031,328903483604537190,4,2022-09-13,2022-09-01,1.0,35.599998,32.631565,10.565565,0.00,2.968435
58199,4565501392096466259,328903483604537190,4,2022-08-10,2022-08-01,1.0,35.599998,32.631565,10.565565,0.00,2.968435
58398,6541172490944157777,328903483604537190,4,2022-09-01,2022-09-01,1.0,35.599998,32.631565,10.565565,0.00,2.968435


# Produto por PDV / SKU

- PDV: ponto de venda: local da venda.
- SKU: código do produto.

Em outras palavras, quantos produtos foram vendidos por PDV (local), para cada SKU (produto).

- internal_store_id = pdv
- internal_product_id = produto